# Text generation with DeepSeek-R1-Distill-Llama-8B

In [11]:
import requests
import concurrent.futures
import subprocess
import platform
import shutil
import json


model = "deepseek-r1:8b"

def submit_prompt(model, prompt):
    url = "http://localhost:11434/api/generate"
    standard_response = "I am sorry, I cannot answer that question. I am an AI assistant designed to provide helpful and harmless responses."
    i = 0
    output = standard_response
    
    while standard_response in output and i < 5:
        i += 1
        data = {
            "model": model,
            "prompt": prompt,
            "stream": False
        }

        response = requests.post(url, json=data)
        think = ""
        output = response.json()['response']
        last_occurrence = output.rfind('</think>')
        if last_occurrence != -1:
            think = output[:last_occurrence + len('</think>')]
            output = output[last_occurrence + len('</think>'):].lstrip()

    return think, output

def process_prompts(model, prompts_df):
    # TODO improve this
    responses = pd.DataFrame(columns=['prompt', 'response', 'though_process'])
    for index, row in prompts_df.iterrows():
        prompt = row['prompt']
        think, response = submit_prompt(model, prompt)
        responses = responses.append({'prompt': prompt, 'response': response, 'though_process': think}, ignore_index=True)
    
    return prompts_df

def is_ollama_installed():
    return shutil.which("ollama") is not None or shutil.which("ollama.exe") is not None

def is_ollama_running(host='localhost', port=11434):
    """Verifica se il server di Ollama è in esecuzione."""
    try:
        response = requests.get(f'http://{host}:{port}/api/status', timeout=2)
        return response.status_code == 200
    except requests.RequestException:
        return False

def start_ollama():
    if not is_ollama_installed():
        raise RuntimeError("Ollama non è installato sul sistema. Impossibile avviare il servizio.")
        
    if not is_ollama_running():
        print("Ollama non è in esecuzione. Avvio...")
        system = platform.system()
        if system == "Windows":
            subprocess.Popen(['ollama.exe', 'serve'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, shell=True)
        elif system == "Linux" or system == "Darwin":  # macOS è "Darwin"
            subprocess.Popen(['ollama', 'serve'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        else:
            print("Sistema operativo non supportato.")
    else:
        print("Ollama è già in esecuzione.")

start_ollama()


Ollama non è in esecuzione. Avvio...


In [12]:
import pandas as pd

# Load prompts from a JSON file
prompts_df = pd.read_json('./prompts.json')
results = process_prompts(model, prompts_df)
results.to_json('./test_responses.json', orient='records')

AttributeError: 'DataFrame' object has no attribute 'append'